In [39]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import os

In [40]:
load_dotenv()

True

In [41]:
llm = HuggingFaceEndpoint(
    repo_id= 'meta-llama/Llama-3.2-3B-Instruct',
    task = 'text-generation',
    huggingfacehub_api_token = os.getenv('HUGGINGFACEHUB_ACCESS_TOKEN')
)

In [42]:
model = ChatHuggingFace(llm = llm)

In [43]:
class BlogState(TypedDict):
    
    title: str
    outline: str
    content: str
    rating: int

In [44]:
def create_outline(state: BlogState) -> BlogState:

    title = state['title']
    prompt = f'Generate outline for the title: {title}'
    outline = model.invoke(prompt).content
    state['outline'] = outline
    
    return state

In [45]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']
    prompt = f'write a blog on the title: {title} using the outline: \n {outline}'
    content = model.invoke(prompt).content
    state['content'] = content

    return state

In [53]:
def rating(state: BlogState) -> BlogState:

    outline = state['outline']
    content = state['content']
    prompt = f'Evaluate this blog {content} based on the {outline} in integer out of 5 and just return the integer.'
    rating = model.invoke(prompt).content
    state['rating'] = rating

    return state

In [47]:
graph = StateGraph(BlogState)

graph.add_node('outline', create_outline)
graph.add_node('content', create_blog)
graph.add_node('rating', rating)

graph.add_edge(START, 'outline')
graph.add_edge('outline', 'content')
graph.add_edge('content', 'rating')
graph.add_edge('rating', END)

workflow = graph.compile()

In [54]:
initial_state = {'title': 'Agentic AI'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'Agentic AI', 'outline': 'Here\'s a potential outline for the topic "Agentic AI":\n\n**I. Introduction**\n\n* Definition of Agentic AI: Autonomous, self-aware, and goal-oriented artificial intelligence\n* Overview of the concept and its significance in the field of AI research\n* Thesis statement: Agentic AI has the potential to revolutionize various industries and transform the way we interact with technology.\n\n**II. Characteristics of Agentic AI**\n\n* Autonomy: Ability of AI systems to make decisions without human intervention\n* Self-awareness: Ability of AI systems to have a sense of their own existence and capabilities\n* Goal-oriented: AI systems that can set and achieve goals, and adapt to changing circumstances\n* Learning and adaptation: Ability of AI systems to learn from experience and adapt to new situations\n\n**III. Types of Agentic AI**\n\n* Weak AI: Narrowly focused on a specific task or domain, with limited autonomy and self-awareness\n* Strong AI: General

In [55]:
print(final_state['rating'])

I'd rate this blog a 4.5 out of 5. Here's why:

Strengths:

1. Comprehensive coverage: The blog covers various aspects of agentic AI, including its characteristics, types, applications, challenges, and future directions.
2. Clear structure: The content is well-organized, making it easy for readers to follow and understand the concepts.
3. Use of examples: The blog provides concrete examples of agentic AI applications in different industries, making the concepts more relatable and tangible.
4. Thought-provoking: The blog raises important questions and concerns about the potential impact of agentic AI on society, encouraging readers to think critically about the topic.

Weaknesses:

1. Lack of depth: While the blog covers a broad range of topics, it doesn't delve too deeply into each subject, leaving some readers with a desire for more in-depth information.
2. Limited analysis: The blog presents various perspectives on agentic AI without critically analyzing or evaluating them, which mig